In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode

import random
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
from PIL import Image
import io
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);
 
      video = document.createElement('video');
      video.setAttribute('playsinline', '');
 
      div.appendChild(video);
 
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;
 
      await video.play();
 
      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
 
      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }
 
    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }
 
    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }
 
  ''')
  display(js)
 
def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)
 
def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [ ]:
VideoCapture()
eval_js('create()')

# 1. Deteccion de rostros

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("/content/drive/My Drive/Intro_procesamiento_de_imagenes/Data")

In [ ]:
VideoCapture()
eval_js('create()')
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  print(im.shape)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  faces = face_detector.detectMultiScale(gray, scaleFactor = 1.1, 4) # de que tanto tamaño va ser la cara a detectar
  print(faces)
  print("hola")
  for (x, y, w, h) in faces:
    #x,y: posiciones
    # w:ancho
    cv2.rectangle(im, (x,y), (x+w,y+h), (255,0,0), 2) # grosor de 2 pixeles

  eval_js('showimg("{}")'.format(image2byte(im)))



In [ ]:
plt.imshow(im[x:x+w,y:y+h])

## 1.1 Difuminacion de rostro

In [ ]:
VideoCapture()
eval_js('create()')
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
 
while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  faces = face_detector.detectMultiScale(gray, 1.1, 4)
  zero = np.zeros(im.shape)


  for (x, y, w, h) in faces:
    kernel = np.ones((50,50),np.float32)/2500
    im[y:y+h,x:x+w] = cv2.filter2D(im[y:y+h,x:x+w],-1,kernel) # aplicamos el filtro a la cara detectada

  eval_js('showimg("{}")'.format(image2byte(im)))

In [ ]:
# haarcascade usa la imagen integral

## 1.2 Recortar la cara

In [ ]:
VideoCapture()
eval_js('create()')
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
 
while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  faces = face_detector.detectMultiScale(gray, 1.1, 4) # te da el x,y,w,h de la cara 
  
  for (x, y, w, h) in faces:
    a = np.ones((h,w))
    #a =im[y:y+h,x:x+w]

  eval_js('showimg("{}")'.format(image2byte(im[y:y+h,x:x+w])))

## 1.3 Ejercicio 3 ( 10 min - 1 punto )

Reemplazar la cara por una imagen RGB

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
img = mpimg.imread('cafe.jpg')

In [ ]:
# Tarea 4: Donde detecte la cara , se debe superponer una imagen cualquiera, la persona al moverse
# delante de la camara , la imagen q la tape tambien se de mover con ella.

# 2. Deteccion de ojos

In [ ]:
VideoCapture()
eval_js('create()')
eye_detector = cv2.CascadeClassifier('haarcascade_eye.xml')
 
while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  eyes = eye_detector.detectMultiScale(gray, 1.1, 4)
  
  for (x, y, w, h) in eyes:
    cv2.rectangle(im, (x,y), (x+w,y+h), (255,0,0), 2)

  eval_js('showimg("{}")'.format(image2byte(im)))

In [ ]:
# Tarea 5
# Solamente detectar los ojos de la cara que esta en frente a la camara 

In [ ]:
# Tarea 6
# Hacer un detector de asteriscos
# Sacar valor medio de cada una de las filas y columnas
# edwinml148@gmail.com